In [2]:
from langchain_ibm import ChatWatsonx, WatsonxEmbeddings, WatsonxLLM

# based on https://til.simonwillison.net/llms/python-react-pattern/

In [ ]:
model_id="ibm/granite-3-8b-instruct"
url="https://us-south.ml.cloud.ibm.com"
apikey="0cZb46prORvZpwwPng_oLDJOxt5SmJwRqzw0gAfZKJ5_"
project_id="2adbc31f-5dc4-4e42-b730-1e5e360727fd"

parameters = {
    "temperature": 0.0,
    "max_tokens": 1000,
}

llm = ChatWatsonx(
    model_id=model_id,
    url=url,
    apikey=apikey,
    project_id=project_id,
    params=parameters,
)

In [4]:
response=llm.invoke("Hello world")
response

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 61, 'total_tokens': 71}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chatcmpl-26cf59c1e7b145ec285dcb149e73fc91', usage_metadata={'input_tokens': 61, 'output_tokens': 10, 'total_tokens': 71})

In [5]:
response.content

'Hello! How can I assist you today?'

In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        print(self.messages)
        completion = llm.invoke(self.messages)
        #completion = llm.invoke(self.messages[0]['content'])
        return completion.content

In [41]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you 
then make sure you PAUSE at this step, and don't write anything else.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Copromptllie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [42]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [43]:
abot = Agent(prompt)
result = abot("How much does a toy poodle weigh?")
print(result)

[{'role': 'system', 'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you \nthen make sure you PAUSE at this step, and don't write anything else.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Copromptllie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 5

In [26]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [27]:
next_prompt = "Observation: {}".format(result)
abot(next_prompt)

[{'role': 'system', 'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you \nthen make sure you PAUSE at this step, and don't write anything else.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Copromptllie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 5

'Thought: The observation provided gives a different weight range than the previous one. I should use the most recent information.\nAction: None\nPAUSE\n\nAnswer: A Toy Poodle weighs 4 to 6 pounds, but according to the most recent information, the average weight is 7 pounds.'

In [ ]:
abot.messages

In [ ]:
abot = Agent(prompt)
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

In [ ]:
abot(next_prompt)